In [1]:
from tensorflow import keras

from tensorflow.keras import models 
from tensorflow.keras import layers 
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 

In [2]:
pd.set_option('mode.chained_assignment', None)

# Preparing the dataset and creating the features and target arrays


n_sims = 10000
n_list = [i for i in range(n_sims)]


h0s = []
results = []


start = 0
end = 10000

x = []
y =[]

for i in n_list[start:end]:
	
	data = np.load(f'input/data_{i+1}.npy')
	x.append(data[:, :2])
	y.append(data[:, 2:][0][0])
	print(f'Status: {((i+1)/end * 100):.2f} %', end='\r')


x = np.array(x)
# x[:,:,0] /= x[:,:,0].max()

y = np.array(y)



kfold = KFold(n_splits=5, shuffle=True)


In [3]:
# K-fold Cross Validation

fold_no = 1

acc_per_fold = []
loss_per_fold = []


for train, test in kfold.split(x, y):

    model = models.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(16))
    model.add(layers.Dense(1))
    model.compile(loss=keras.losses.MeanSquaredError(), optimizer='adam', metrics=keras.metrics.MeanAbsoluteError())


    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')


    history = model.fit(x[train], y[train], epochs=25, batch_size=10, verbose=0)


    scores = model.evaluate(x[test], y[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}\n')
    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no += 1

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 0.006206299178302288; mean_absolute_error of 0.06511002033948898

------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 0.07019098103046417; mean_absolute_error of 0.2580276131629944

------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 0.07935651391744614; mean_absolute_error of 0.27942559123039246

------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 2.172879695892334; mean_absolute_error of 1.470821499824524

------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 0.7708052396774292; mean_absolute_error of 0.8741707801818848



In [4]:
np.mean(loss_per_fold)

0.6198877459391952

In [5]:
np.mean(acc_per_fold)

0.5895111009478569